In [1]:
import os
import sys
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import re

sys.path.append("../")
from helpers.get_density import get_density
from helpers.get_spins import get_spins
from helpers.downscale import downscale_3d
from plot_multiple_3Ds import plot_multiple_3Ds
from loader import get_snap_paths, load_Psi
from helpers.grids import get_kw_square_nonzero_grid, get_radius_grid, get_space_grid
from multiprocessing import Pool
import plotly.graph_objs as go

In [2]:
box_N = 96
box_L = 20.0
box_dx = box_L / box_N

In [3]:
lambdas = [0]
runs = [1]
dtos = [1, 2, 4, 8]
to_loads = []
for ld in lambdas:
	for run in runs:
		sr = {}
		for dto in dtos:
			sim_path = f"../out_remote/2022-09-20/2-solitons,method2,lambda={ld},run={run},dto={dto}/"
			snap_paths = get_snap_paths(sim_path, box_L, box_N)
			for (idx, (itr, time, path)) in enumerate(snap_paths):
				# if time != 0 and idx % 2 == 0:
				if time != 0:
					to_loads.append((ld, run, dto, itr, time, path))


In [4]:
def load_one(x):
	(ld, run, dto, itr, time, path) = x
	try:
		Psi = load_Psi(path)
		return (ld, run, dto, itr, time, Psi)
	except:
		print(f"{path} load failed")
		return (ld, run, dto, itr, time, None)

with Pool(32) as p:
 	loadeds = p.map(load_one, to_loads)

loading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-500-10.85.matloading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-700-15.19.matloading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-600-13.02.matloading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-300-6.51.matloading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-200-4.34.matloading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-1000-21.70.matloading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-800-17.36.matloading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-100-2.17.matloading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-1200-26.04.matloading ../out_remote/2022-09-20/2-solitons,method2,lambda=0,run=1,dto=1/snap-Psi-1600-34.72.matloading ../out_remote/2022-09-20/2-solitons,method

In [5]:
def test_for(test_ld, test_run, line_col, area_col, dtos=dtos):
	snaps = {dto: [] for dto in dtos}
	for (ld, run, dto, itr, time, Psi3) in sorted(loadeds):
		if (test_ld == ld and test_run == run and dto in snaps):
			snaps[dto].append((time, Psi3))
	
	def dif_fn(Psi3_1, Psi3_2):
		# res =  np.abs(get_density(Psi3_1) - get_density(Psi3_2))
		res =  np.sqrt(get_density(Psi3_1 - Psi3_2))
		# res = np.sqrt(get_density(get_spins(Psi3_1) - get_spins(Psi3_2)))
		return res
	# rhos = {dto: Psi3s for (dto, Psi3s) in snaps.items()}

	xs = []
	y1s = []
	# y2s = []
	# y3s = []
	for ((t1, p1), (t2, p2), (t4, p4)) in zip(snaps[dtos[0]], snaps[dtos[1]], snaps[dtos[2]]):
		assert(t1 == t2 == t4)
		s1 = np.sum(get_density(p1))
		s2 = np.sum(get_density(p2))
		s4 = np.sum(get_density(p4))
		# s1 = np.sum(get_spins(p1), axis=(1, 2, 3))
		# s2 = np.sum(get_spins(p2), axis=(1, 2, 3))
		# s4 = np.sum(get_spins(p4), axis=(1, 2, 3))
		v = np.abs(s1 - s2) / np.abs(s2 - s4)
		q = np.log2(v)
		xs.append(t1)
		y1s.append(q)
		# y1s.append(q[0])
		# y2s.append(q[1])
		# y3s.append(q[2])
	fig = [
		pgo.Scatter(
			x=xs,
			y=y1s,
			line=dict(color=line_col),
			mode='lines'
		),
		# pgo.Scatter(
		# 	x=xs,
		# 	y=y2s,
		# 	line=dict(color=line_col),
		# 	mode='lines'
		# ),
		# pgo.Scatter(
		# 	x=xs,
		# 	y=y3s,
		# 	line=dict(color=line_col),
		# 	mode='lines'
		# )
	]

	# xs = []
	# y05s = []
	# y25s = []
	# y50s = []
	# y75s = []
	# y95s = []
	# for ((t1, p1), (t2, p2), (t4, p4)) in zip(snaps[dtos[0]], snaps[dtos[1]], snaps[dtos[2]]):
	# 	assert(t1 == t2 == t4)
	# 	v = dif_fn(p1, p2) / dif_fn(p2, p4)
	# 	q = np.log2(v)

	# 	xs.append(t1)
	# 	pcs = np.percentile(q, [5, 25, 50, 75, 95])
	# 	for (ys, y) in zip([y05s, y25s, y50s, y75s, y95s], pcs):
	# 		ys.append(y)
	# fig = [
	# 	pgo.Scatter(
	# 		x=xs,
	# 		y=y50s,
	# 		line=dict(color=line_col),
	# 		mode='lines'
	# 	),
	# 	pgo.Scatter(
	# 		x=xs+xs[::-1],
	# 		y=y05s+y95s[::-1],
	# 		fill='toself',
	# 		fillcolor=area_col,
	# 		line=dict(color='rgba(255,255,255,0)'),
	# 		hoverinfo="skip",
	# 		# showlegend=False
	# 	),
	# 	pgo.Scatter(
	# 		x=xs+xs[::-1],
	# 		y=y25s+y75s[::-1],
	# 		fill='toself',
	# 		fillcolor=area_col,
	# 		line=dict(color='rgba(255,255,255,0)'),
	# 		hoverinfo="skip",
	# 		# showlegend=False
	# 	)	
	# ]
	return fig
# fig += test_for(0, 1, 'rgb(26, 188, 156)', 'rgba(26, 188, 156, 0.2)')
# fig += test_for(-1, 1, 'rgb(52, 152, 219)', 'rgba(52, 152, 219, 0.2)')
# fig += test_for(1, 1, 'rgb(243, 156, 18)', 'rgba(243, 156, 18, 0.2)')

In [6]:
fig = []
fig += test_for(0, 1, 'rgb(26, 188, 156)', 'rgba(26, 188, 156, 0.2)', [1, 2, 4])
fig = pgo.Figure(fig)
fig.write_html("2022-09-21-plot_convergence-N-method2,124,.html")

fig = []
fig += test_for(0, 1, 'rgb(26, 188, 156)', 'rgba(26, 188, 156, 0.2)', [2, 4, 8])
fig = pgo.Figure(fig)
fig.write_html("2022-09-21-plot_convergence-N-method2,248.html")